In [324]:
import csv
import pandas as pd
import numpy as np

In [268]:
path = './data/game_data_public.MKM.PremierDraft.csv'
with open(path) as csvfile:
    base_df = pd.read_csv(csvfile)

In [302]:
df = pd.DataFrame(base_df)

In [303]:
all_columns = list(df.columns)
name_columns = [col for col in all_columns if 'drawn_' in col]
names = [n.split('_')[1] for n in name_columns]

name_gen = lambda x: [f'{x}_{name}' for name in names]

In [373]:
oh_df = df[name_gen('opening_hand')]
oh_df.columns = names

drawn_df = df[name_gen('drawn')]
drawn_df.columns = names

ih_df = oh_df + drawn_df

In [374]:
cards_seen = ih_df.sum(axis=1)
cards_seen.name = 'cards_seen'

cards_not_seen = cards_seen.apply(lambda x: max(1, 40-x))
cards_not_seen.name = 'cards_not_seen'

In [375]:
gih_counts = ih_df.sum()
gih_win_counts = ih_df[df['won']].sum()

gihwr = gih_win_counts / gih_counts
gihwr.name = 'gihwr'

In [376]:
deck_df = pd.DataFrame(df[name_gen('deck')])
deck_df.columns = names

deck_counts = deck_df.sum()
deck_win_counts = deck_df[df['won']].sum()

gpwr = deck_win_counts / deck_counts
gpwr.name = 'gpwr'

In [377]:
ns_df = deck_df - ih_df
ns_df = ns_df.apply(lambda S: S.apply(lambda x: max(0, x)))

In [378]:
gns_counts = ns_df.sum()
gns_win_counts = ns_df[df['won']].sum()

gnswr = gns_win_counts / gns_counts
gnswr.name = 'gnswr'

In [379]:
ihd = gihwr - gpwr
ihd.name = 'in-hand delta'

In [380]:
gwih = ih_df.div(cards_seen, axis=0) * 40
gwns = ns_df.div(cards_not_seen, axis=0) * 40

In [381]:
gwihwr = gwih[df['won']].sum() / gwih.sum()
gwihwr.name = 'gwihwr'

In [382]:
gwihd = gwihwr - gpwr
gwihd.name = 'game-weighted in-hand delta'

In [383]:
deck_adjacency = deck_df.transpose().dot(deck_df)
average_marginal_decklist = deck_adjacency / deck_adjacency.sum()
d1gpwr = gpwr.dot(average_marginal_decklist)

d1d = gpwr - d1gpwr
d1d.name = 'distance-one delta'

In [384]:
ihd_df = pd.DataFrame([ihd, gwihd, d1d])

In [385]:
print(d1d.describe())
print(ihd.describe())
gwihd.describe()

count    326.000000
mean      -0.016097
std        0.033482
min       -0.198801
25%       -0.027297
50%       -0.009988
75%        0.005723
max        0.052500
Name: d1delta, dtype: float64
count    326.000000
mean       0.003434
std        0.025944
min       -0.130455
25%       -0.007860
50%        0.002450
75%        0.014836
max        0.077208
Name: ihd, dtype: float64


count    326.000000
mean      -0.002477
std        0.029189
min       -0.180132
25%       -0.015249
50%       -0.003459
75%        0.012070
max        0.077607
Name: gwihd, dtype: float64

Index(['expansion', 'event_type', 'draft_id', 'draft_time', 'game_time',
       'build_index', 'match_number', 'game_number', 'rank', 'opp_rank',
       'main_colors', 'splash_colors', 'on_play', 'num_mulligans',
       'opp_num_mulligans', 'opp_colors', 'num_turns', 'won',
       'opening_hand_A Killer Among Us', 'drawn_A Killer Among Us'],
      dtype='object')

In [391]:
turn_info_df = df[['num_turns', 'won']]
speed_bias_df = pd.concat([deck_df, turn_info_df], axis=1)

In [392]:
dot_df = turn_info_df.transpose().dot(speed_bias_df)

In [394]:
dot_df

,A Killer Among Us,Absolving Lammasu,Aftermath Analyst,Agency Coroner,Agency Outfitter,"Agrus Kos, Spirit of Justice",Airtight Alibi,Alley Assailant,"Alquist Proft, Master Sleuth",Analyze the Pollen,...,Vitu-Ghazi Inspector,Warleader's Call,Wispdrinker Vampire,Wojek Investigator,Worldsoul's Rage,Worldspine Wurm,Wrench,"Yarus, Roar of the Old Gods",num_turns,won
num_turns,844605,220869,406587,128508,114414,143957,411017,630815,125199,225370,...,1311022,271571,384519,327424,117544,4623,152064,242758,58545442,3164147
won,52963,12841,22491,6692,6115,9609,24024,34714,7299,13141,...,75457,18119,21808,20770,6654,221,8888,14559,3164147,346373
